<center> <h1>Algorithms in Computational Biology (INFO-F438)</h1> </center>

<center> <h2>Assignment 3: Efficient String Matching</h2> </center>
 
<center> <h3>Guillaume Buisson-Chavot</h3> </center> 
<center> <h3>matricule: 000465822 </h3> </center> 

To test the algorithm, I wrote a random text which contains the different keywords from the article:

In [125]:
text = "he is very sweet. She would like hers socks and he would like his ice"
keywords = ['he', 'she', 'his', 'hers']

total_string = text
for word in keywords:
    total_string += word
alphabet = {letter for letter in total_string}
print('alphabet = ', alphabet)

alphabet =  {' ', 't', 'y', '.', 'i', 'w', 'h', 'c', 'S', 'n', 'e', 'a', 'v', 's', 'r', 'u', 'l', 'k', 'o', 'd'}


In [126]:
def goto_create(list_of_keywords):
    '''
    construction of the goto graph and a part of the output function
    input: list of strings because a set is unordered in python,
    that means that we can't use indexes to search in a set. And
    it's possible in a list
    output: goto and output which are dictionaries, goto represents the
    goto graph and output corresponds to the partially computed output
    function
    '''
    output, goto, newstate = {}, {}, 0
    for word in list_of_keywords: 
        state, j = 0, 0
        while (state,word[j]) in goto:
            state = goto[(state,word[j])]
            j += 1
        for p in range(j,len(word)):
            newstate += 1
            goto[(state,word[p])] = newstate
            state = newstate
        output[state] = [word]
    for a in alphabet:
        if (0, a) not in goto:
            goto[(0,a)] = 0
    return goto, output


In [127]:
print("Test of goto function with set_of_keywords = {'he', 'she', 'his', 'hers'}:")

goto, output = goto_create(keywords)
print('goto function =', goto)
print('partially computed output function =', output)

Test of goto function with set_of_keywords = {'he', 'she', 'his', 'hers'}:
goto function = {(0, 'h'): 1, (1, 'e'): 2, (0, 's'): 3, (3, 'h'): 4, (4, 'e'): 5, (1, 'i'): 6, (6, 's'): 7, (2, 'r'): 8, (8, 's'): 9, (0, ' '): 0, (0, 't'): 0, (0, 'y'): 0, (0, '.'): 0, (0, 'i'): 0, (0, 'w'): 0, (0, 'c'): 0, (0, 'S'): 0, (0, 'n'): 0, (0, 'e'): 0, (0, 'a'): 0, (0, 'v'): 0, (0, 'r'): 0, (0, 'u'): 0, (0, 'l'): 0, (0, 'k'): 0, (0, 'o'): 0, (0, 'd'): 0}
partially computed output function = {2: ['he'], 5: ['she'], 7: ['his'], 9: ['hers']}


In [128]:
def create_failure(goto, output):
    '''
    function which creates the failure function and updates the output function
    input: goto and partially computed output dictionaries
    output: failure and output dictionaries
    '''
    failure = {}
    queue = []
    #filling out the failure dictionary with the first values = 0
    for a in alphabet:
        s = goto[(0,a)] 
        if s != 0:
            queue.append(s)
            failure[s] = 0
    #filling out the failure dictionary with the other values from the first values        
    while queue != []:
        r = queue.pop(0) 
        for a in alphabet:
            if (r,a) in goto:
                s = goto[(r,a)]
                queue.append(s)             
                state = failure[r]
                while (state, a) not in goto:
                    state = failure[state]
                failure[s] = goto[(state,a)]  
    #update of the output dictionary if (s) and failure[s] are keys of the output dictionary
                if (s) in output and (failure[s]) in output:
                    for i in range(len(output[failure[s]])):
                        output[s].append(output[failure[s]][i])
    return failure, output

In [129]:
failure, output = create_failure(goto, output)
print(failure)
print(output)

{1: 0, 3: 0, 6: 0, 2: 0, 4: 1, 7: 3, 8: 0, 5: 2, 9: 3}
{2: ['he'], 5: ['she', 'he'], 7: ['his'], 9: ['hers']}


In [130]:
def pattern_matching_machine(text, goto, failure, output):
    '''
    function which searches the locations of the keywords in a text from goto, failure
    and output dictionaries built previously
    input: text which is a string, goto, failure and output which are dictionaries
    output: locations which is a dictionary, keys are the last index of the keywords 
    and values are the correspondant keywords found in the text
    '''
    locations, state = {}, 0
    for i in range(len(text)):
        while (state, text[i]) not in goto:
            state = failure[state]
        state = goto[(state, text[i])]
        if (state) in output:
            locations[i] = output[state]
    return locations

In [131]:
locations = pattern_matching_machine(text, goto, failure, output)
print("reminder:","\ntext = 'he is very sweet. She would like hers socks and he would like his ice'", 
      "\nkeywords = ['he', 'she', 'his', 'hers']")
print('locations =', locations)

reminder: 
text = 'he is very sweet. She would like hers socks and he would like his ice' 
keywords = ['he', 'she', 'his', 'hers']
locations = {1: ['he'], 20: ['he'], 34: ['he'], 36: ['hers'], 49: ['he'], 64: ['his']}


In [132]:
def next_move_create(goto, failure):
    '''
    function which allows to eliminate all failure transition in pattern_matching_machine function.
    For each state s and input letter or symbol a, delta[(s,a)] is a state in a finite set of states S. This 
    function is built by using goto and failure dictionaries and precomputes the result of every 
    sequence of possible failure transitions
    input: goto and failure dictionaries
    output: delta which is a dictionary
    '''
    queue = []
    delta = {}
    for a in alphabet:                                   # for each symbol a do
        delta[(0, a)] = goto[(0, a)]                     # delta(0,a) <- g(0,a)
        if goto[(0, a)] != 0:                            # if g(0,a) != 0 then
            queue.append(goto[(0, a)])                   # queue <- queue U {g(0,a)}
    while queue != []:                                     # while queue != empty do
        r = queue.pop(0)                                 # let r be the next state in queue
                                                         # queue <- queue - {r}
        for a in alphabet:                               # for each symbol a do
            if (r, a) in goto:                           # if g(r,a) = s != fail do
                s = goto[(r, a)]
                queue.append(s)                          # queue <- queue U {s}
                delta[(r, a)] = s                        # delta(r,a) <- s
            else:                                        # else
                delta[(r, a)] = delta[(failure[r], a)]   # delta(r,a) <- delta(f(r),a)
    return delta

In [133]:
def pattern_matching_machine2(text, output):
    '''
    function which searches the locations of the keywords in a text from delta
    and output dictionaries built previously
    input: text which is a string and output which is a dictionary
    output: locations which is a dictionary, keys are the last index of the keywords 
    and values are the correspondant keywords found in the text
    '''
    locations, state = {}, 0
    for i in range(len(text)):
        state = delta[(state, text[i])]
        if (state) in output:
            locations[i] = output[state]
    return locations

In [134]:
delta = next_move_create(goto, failure)
print('delta= \n', delta)
print(120*'-')
locations = pattern_matching_machine2(text, output)
print("reminder:","\ntext = 'he is very sweet. She would like hers socks and he would like his ice'", 
      "\nkeywords = ['he', 'she', 'his', 'hers']")
print('locations: ', locations)

delta= 
 {(0, ' '): 0, (0, 't'): 0, (0, 'y'): 0, (0, '.'): 0, (0, 'i'): 0, (0, 'w'): 0, (0, 'h'): 1, (0, 'c'): 0, (0, 'S'): 0, (0, 'n'): 0, (0, 'e'): 0, (0, 'a'): 0, (0, 'v'): 0, (0, 's'): 3, (0, 'r'): 0, (0, 'u'): 0, (0, 'l'): 0, (0, 'k'): 0, (0, 'o'): 0, (0, 'd'): 0, (1, ' '): 0, (1, 't'): 0, (1, 'y'): 0, (1, '.'): 0, (1, 'i'): 6, (1, 'w'): 0, (1, 'h'): 1, (1, 'c'): 0, (1, 'S'): 0, (1, 'n'): 0, (1, 'e'): 2, (1, 'a'): 0, (1, 'v'): 0, (1, 's'): 3, (1, 'r'): 0, (1, 'u'): 0, (1, 'l'): 0, (1, 'k'): 0, (1, 'o'): 0, (1, 'd'): 0, (3, ' '): 0, (3, 't'): 0, (3, 'y'): 0, (3, '.'): 0, (3, 'i'): 0, (3, 'w'): 0, (3, 'h'): 4, (3, 'c'): 0, (3, 'S'): 0, (3, 'n'): 0, (3, 'e'): 0, (3, 'a'): 0, (3, 'v'): 0, (3, 's'): 3, (3, 'r'): 0, (3, 'u'): 0, (3, 'l'): 0, (3, 'k'): 0, (3, 'o'): 0, (3, 'd'): 0, (6, ' '): 0, (6, 't'): 0, (6, 'y'): 0, (6, '.'): 0, (6, 'i'): 0, (6, 'w'): 0, (6, 'h'): 1, (6, 'c'): 0, (6, 'S'): 0, (6, 'n'): 0, (6, 'e'): 0, (6, 'a'): 0, (6, 'v'): 0, (6, 's'): 7, (6, 'r'): 0, (6, 'u'): 0, (6

Now, I will test the algorithm with the text of description and the following keywords: pattern, tree, state, prove,
the, it.

In [135]:
text = "As discussed in the session on Combinatorial Pattern Matching, keyword trees provide an efficient solution to search for multiple k patterns in a text of length n. The algorithm requires first the construction of a keyword tree and then, using naïve threading, the patterns can be identified in O(nm), where n is the average length of the k patterns and m is the length of the text. Alfred Aho and Margaret Corasick proposed in 1975 a more efficient solution that allows one to identify the patterns in O(m) time. To achieve this improvement, the keyword tree is replaced by a finite state pattern matching automata. Once this machine is constructed, the text can be processed and the starting positions for the different patterns can be returned as output. The full specification of the Aho-Corasick algorithm is provided in the original article included with this assignment."
keywords = ['pattern', 'tree', 'state', 'prove', 'the', 'it']

#creation of the alphabet from the text and keywords
total_string = text
for word in keywords:
    total_string += word 
alphabet = {letter for letter in total_string}
print('alphabet = ', alphabet)
print(120*'-')

#test of the first pattern_matching_machine
goto, output = goto_create(keywords)
failure, output = create_failure(goto, output)
locations = pattern_matching_machine(text, goto, failure, output)
print('locations =', locations)
print(120*'-')

#test of the second pattern_matching_machine which uses the next_move function
delta = next_move_create(goto, failure)
locations = pattern_matching_machine2(text, output)
print('locations: ', locations)

alphabet =  {'.', 'c', 'k', 'o', 'd', '-', 'p', '1', 'b', 'q', 'O', 'i', 'h', 'M', 'e', 'v', 'ï', 'u', 'm', 'A', 'g', '(', 'x', 't', '5', 'C', 'P', ')', 'a', 's', 'r', 'l', ',', ' ', 'y', 'w', 'f', 'n', '9', '7', 'T'}
------------------------------------------------------------------------------------------------------------------------
locations = {18: ['the'], 74: ['tree'], 138: ['pattern'], 174: ['it'], 195: ['the'], 226: ['tree'], 234: ['the'], 263: ['the'], 271: ['pattern'], 315: ['the'], 337: ['the'], 347: ['pattern'], 361: ['the'], 375: ['the'], 489: ['the'], 497: ['pattern'], 536: ['prove'], 545: ['the'], 558: ['tree'], 581: ['it'], 588: ['state'], 596: ['pattern'], 653: ['the'], 683: ['the'], 698: ['it'], 710: ['the'], 728: ['pattern'], 786: ['the'], 807: ['it'], 828: ['the'], 858: ['it']}
------------------------------------------------------------------------------------------------------------------------
locations:  {18: ['the'], 74: ['tree'], 138: ['pattern'], 174: ['it']